In [1]:
import numpy as np

import tensorflow as tf

/Users/adam/anaconda3/envs/energy_py/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/adam/anaconda3/envs/energy_py/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
obs = tf.placeholder(shape=(None, 5), dtype=tf.float32)
next_obs = tf.placeholder(shape=(None, 5), dtype=tf.float32)

o_p = np.arange(5).reshape(1, 5)
no_p = np.arange(5).reshape(1, 5)

In [3]:
def make_layer(scope, 
               input_tensor, 
               input_shape, 
               output_nodes):
    
    input_weights = tf.get_variable(
        'weights',
        shape=(5, output_nodes),
        initializer=tf.contrib.layers.xavier_initializer()
    )

    input_bias = tf.get_variable(
        'input_bias',
        shape=(output_nodes),
        initializer=tf.zeros_initializer()
    )

    layer = tf.add(
        tf.matmul(input_tensor, input_weights),
        input_bias,
        name='input_layer'
    )
        
    return tf.nn.relu(layer)

In [4]:
with tf.variable_scope('online_networks') as scope:
    online = make_layer('online', obs, (5,), 10)
    
    scope.reuse_variables()
    
    online_double_q = make_layer('online_double_q', next_obs, (5,), 10)

target = make_layer('target', obs, (5,), 10)

Instructions for updating:
Use the retry module or similar alternatives.


In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    o = sess.run(online, {obs: o_p})
    
    d = sess.run(online_double_q, {next_obs: no_p})
    
    t = sess.run(target, {obs: o_p})

In [6]:
o

array([[0.        , 0.        , 1.5633994 , 0.        , 0.        ,
        0.        , 1.4100325 , 0.        , 1.789311  , 0.94808316]],
      dtype=float32)

In [7]:
d

array([[0.        , 0.        , 1.5633994 , 0.        , 0.        ,
        0.        , 1.4100325 , 0.        , 1.789311  , 0.94808316]],
      dtype=float32)

In [8]:
t

array([[0.        , 0.        , 1.8985631 , 0.2113083 , 3.6537354 ,
        0.17140895, 0.        , 0.        , 1.0960298 , 0.7441188 ]],
      dtype=float32)